In [2]:
config = {
    "database_path": "/nas/rishith/datasets/asvSpoof2019/DS_10283_3336/LA/",
    "asv_score_path": "ASVspoof2019_LA_asv_scores/ASVspoof2019.LA.asv.eval.gi.trl.scores.txt",
    "model_path": "./models/weights/AASIST.pth",
    "batch_size": 24,
    "num_epochs": 100,
    "loss": "CCE",
    "track": "LA",
    "eval_all_best": "True",
    "eval_output": "eval_scores_using_best_dev_model.txt",
    "cudnn_deterministic_toggle": "True",
    "cudnn_benchmark_toggle": "False",
    "model_config": {
        "architecture": "AASIST",
        "nb_samp": 64600,
        "first_conv": 128,
        "filts": [70, [1, 32], [32, 32], [32, 64], [64, 64]],
        "gat_dims": [64, 32],
        "pool_ratios": [0.5, 0.7, 0.5, 0.5],
        "temperatures": [2.0, 2.0, 100.0, 100.0]
    },
    "optim_config": {
        "optimizer": "adam", 
        "amsgrad": "False",
        "base_lr": 0.0001,
        "lr_min": 0.000005,
        "betas": [0.9, 0.999],
        "weight_decay": 0.0001,
        "scheduler": "cosine"
    }
}

In [3]:
# load experiment configurations
# with open(args.config, "r") as f_json:
#     config = json.loads(f_json.read())
model_config = config["model_config"]
optim_config = config["optim_config"]
optim_config["epochs"] = config["num_epochs"]
track = config["track"]
assert track in ["LA", "PA", "DF"], "Invalid track given"
if "eval_all_best" not in config:
    config["eval_all_best"] = "True"
if "freq_aug" not in config:
    config["freq_aug"] = "False"

In [4]:
import json
import os
import sys
import warnings
from importlib import import_module
from pathlib import Path
from shutil import copy
from typing import Dict, List, Union

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from torchcontrib.optim import SWA

from data_utils import (Dataset_ASVspoof2019_train,
                        Dataset_ASVspoof2019_devNeval, genSpoof_list)
from evaluation import calculate_tDCF_EER
from utils import create_optimizer, seed_worker, set_seed, str_to_bool

2023-05-29 12:52:45.904506: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-29 12:52:47.521818: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-29 12:52:48.203401: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-05-29 12:52:48.203455: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if yo

In [6]:
# make experiment reproducible
seed = 1234
set_seed(seed, config)

In [7]:
# define database related paths
output_dir = './exp_result'
output_dir = Path(output_dir)
prefix_2019 = "ASVspoof2019.{}".format(track)
database_path = Path(config["database_path"])
dev_trial_path = (database_path /
                      "ASVspoof2019_{}_cm_protocols/{}.cm.dev.trl.txt".format(
                          track, prefix_2019))
eval_trial_path = (
        database_path /
        "ASVspoof2019_{}_cm_protocols/{}.cm.eval.trl.txt".format(
            track, prefix_2019))

In [20]:
# os.path.basename('./config/AASIST.conf')
os.path.splitext(os.path.basename('./config/AASIST.conf'))

('AASIST', '.conf')

In [22]:
# define model related paths
model_tag = "{}_{}_ep{}_bs{}".format(
        track,
        'AASIST',
        config["num_epochs"], config["batch_size"])
# if args.comment:
#     model_tag = model_tag + "_{}".format(acomment)
model_tag = output_dir / model_tag
model_save_path = model_tag / "weights"
eval_score_path = model_tag / config["eval_output"]
writer = SummaryWriter(model_tag)
os.makedirs(model_save_path, exist_ok=True)
# copy(config, model_tag / "config.conf")

In [9]:
# set device
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device: {}".format(device))

Device: cpu


In [11]:
from main import *

In [12]:
# define model architecture
model = get_model(model_config, device)

no. model params:297866


In [13]:
# define dataloaders
trn_loader, dev_loader, eval_loader = get_loader(
        database_path, seed, config)

no. training files: 25380
no. validation files: 24844


In [23]:
# evaluates pretrained model and exit script
model.load_state_dict(
        torch.load(config["model_path"], map_location=device))
print("Model loaded : {}".format(config["model_path"]))
print("Start evaluation...")
produce_evaluation_file(eval_loader, model, device,
                                eval_score_path, eval_trial_path)
calculate_tDCF_EER(cm_scores_file=eval_score_path,
                           asv_score_file=database_path /
                           config["asv_score_path"],
                           output_file=model_tag / "t-DCF_EER.txt")
print("DONE.")
eval_eer, eval_tdcf = calculate_tDCF_EER(
            cm_scores_file=eval_score_path,
            asv_score_file=database_path / config["asv_score_path"],
            output_file=model_tag/"loaded_model_t-DCF_EER.txt")
        

Model loaded : ./models/weights/AASIST.pth
Start evaluation...
Scores saved to exp_result/LA_AASIST_ep100_bs24/eval_scores_using_best_dev_model.txt

CM SYSTEM
	EER		= 0.829511226 % (Equal error rate for countermeasure)

TANDEM
	min-tDCF		= 0.027529478

BREAKDOWN CM SYSTEM
	EER A07		= 0.529676029 % (Equal error rate for A07
	EER A08		= 0.424416206 % (Equal error rate for A08
	EER A09		= 0.000000000 % (Equal error rate for A09
	EER A10		= 0.855630509 % (Equal error rate for A10
	EER A11		= 0.179950347 % (Equal error rate for A11
	EER A12		= 0.709626376 % (Equal error rate for A12
	EER A13		= 0.146004133 % (Equal error rate for A13
	EER A14		= 0.162977240 % (Equal error rate for A14
	EER A15		= 0.553447233 % (Equal error rate for A15
	EER A16		= 0.651908959 % (Equal error rate for A16
	EER A17		= 1.263073608 % (Equal error rate for A17
	EER A18		= 2.607635837 % (Equal error rate for A18
	EER A19		= 0.651908959 % (Equal error rate for A19
DONE.

CM SYSTEM
	EER		= 0.829511226 % (Equal error

# AASIST-L

In [11]:
config = {
    "database_path": "/DATA/nfsshare/rishith/datasets/asvSpoof2019/DS_10283_3336/LA/",
    "asv_score_path": "ASVspoof2019_LA_asv_scores/ASVspoof2019.LA.asv.eval.gi.trl.scores.txt",
    "model_path": "./models/weights/AASIST-L.pth",
    "batch_size": 24,
    "num_epochs": 20,
    "loss": "CCE",
    "track": "LA",
    "eval_all_best": "True",
    "eval_output": "eval_scores_using_best_dev_model.txt",
    "cudnn_deterministic_toggle": "True",
    "cudnn_benchmark_toggle": "False",
    "model_config": {
        "architecture": "AASIST",
        "nb_samp": 64600,
        "first_conv": 128,
        "filts": [70, [1, 32], [32, 32], [32, 24], [24, 24]],
        "gat_dims": [24, 32],
        "pool_ratios": [0.4, 0.5, 0.7, 0.5],
        "temperatures": [2.0, 2.0, 100.0, 100.0]
    },
    "optim_config": {
        "optimizer": "adam", 
        "amsgrad": "False",
        "base_lr": 0.0001,
        "lr_min": 0.000005,
        "betas": [0.9, 0.999],
        "weight_decay": 0.0001,
        "scheduler": "cosine"
    }
}


In [12]:
model_config = config["model_config"]
optim_config = config["optim_config"]
optim_config["epochs"] = config["num_epochs"]
track = config["track"]
assert track in ["LA", "PA", "DF"], "Invalid track given"
if "eval_all_best" not in config:
    config["eval_all_best"] = "True"
if "freq_aug" not in config:
    config["freq_aug"] = "False"

In [13]:
import json
import os
import sys
import warnings
from importlib import import_module
from pathlib import Path
from shutil import copy
from typing import Dict, List, Union

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from torchcontrib.optim import SWA

from data_utils import (Dataset_ASVspoof2019_train,
                        Dataset_ASVspoof2019_devNeval, genSpoof_list)
from evaluation import calculate_tDCF_EER
from utils import create_optimizer, seed_worker, set_seed, str_to_bool

In [14]:
# make experiment reproducible
seed = 1234
set_seed(seed, config)

In [15]:
# define database related paths
output_dir = './exp_result'
output_dir = Path(output_dir)
prefix_2019 = "ASVspoof2019.{}".format(track)
database_path = Path(config["database_path"])
dev_trial_path = (database_path /
                      "ASVspoof2019_{}_cm_protocols/{}.cm.dev.trl.txt".format(
                          track, prefix_2019))
eval_trial_path = (
        database_path /
        "ASVspoof2019_{}_cm_protocols/{}.cm.eval.trl.txt".format(
            track, prefix_2019))

In [16]:
# define model related paths
model_tag = "{}_{}_ep{}_bs{}".format(
        track,
        'AASIST-L',
        config["num_epochs"], config["batch_size"])
# if args.comment:
#     model_tag = model_tag + "_{}".format(acomment)
model_tag = output_dir / model_tag
model_save_path = model_tag / "weights"
eval_score_path = model_tag / config["eval_output"]
writer = SummaryWriter(model_tag)
os.makedirs(model_save_path, exist_ok=True)

In [17]:
# set device
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device: {}".format(device))

Device: cuda


In [18]:
from main import *

In [19]:
# define model architecture
model = get_model(model_config, device)

no. model params:85306


In [20]:
model_config

{'architecture': 'AASIST',
 'nb_samp': 64600,
 'first_conv': 128,
 'filts': [70, [1, 32], [32, 32], [32, 24], [24, 24]],
 'gat_dims': [24, 32],
 'pool_ratios': [0.4, 0.5, 0.7, 0.5],
 'temperatures': [2.0, 2.0, 100.0, 100.0]}

In [21]:
# define dataloaders
trn_loader, dev_loader, eval_loader = get_loader(
        database_path, seed, config)

no. training files: 25380
no. validation files: 24844


In [22]:
# evaluates pretrained model and exit script
model.load_state_dict(
        torch.load(config["model_path"], map_location=device))
print("Model loaded : {}".format(config["model_path"]))
print("Start evaluation...")
produce_evaluation_file(eval_loader, model, device,
                                eval_score_path, eval_trial_path)
calculate_tDCF_EER(cm_scores_file=eval_score_path,
                           asv_score_file=database_path /
                           config["asv_score_path"],
                           output_file=model_tag / "t-DCF_EER.txt")
print("DONE.")
eval_eer, eval_tdcf = calculate_tDCF_EER(
            cm_scores_file=eval_score_path,
            asv_score_file=database_path / config["asv_score_path"],
            output_file=model_tag/"loaded_model_t-DCF_EER.txt")
        

Model loaded : ./models/weights/AASIST-L.pth
Start evaluation...
Scores saved to exp_result/LA_AASIST-L_ep20_bs24/eval_scores_using_best_dev_model.txt

CM SYSTEM
	EER		= 0.990923080 % (Equal error rate for countermeasure)

TANDEM
	min-tDCF		= 0.030805977

BREAKDOWN CM SYSTEM
	EER A07		= 0.553447233 % (Equal error rate for A07
	EER A08		= 0.203721550 % (Equal error rate for A08
	EER A09		= 0.000000000 % (Equal error rate for A09
	EER A10		= 1.001634642 % (Equal error rate for A10
	EER A11		= 0.139206037 % (Equal error rate for A11
	EER A12		= 0.692653269 % (Equal error rate for A12
	EER A13		= 0.203721550 % (Equal error rate for A13
	EER A14		= 0.057717417 % (Equal error rate for A14
	EER A15		= 0.570420339 % (Equal error rate for A15
	EER A16		= 0.634935852 % (Equal error rate for A16
	EER A17		= 1.914982568 % (Equal error rate for A17
	EER A18		= 3.049025150 % (Equal error rate for A18
	EER A19		= 0.709626376 % (Equal error rate for A19
DONE.

CM SYSTEM
	EER		= 0.990923080 % (Equal er